# ETL to Darwin Core

In this notebook, the mapping from the Gull bird tracking to a Darwin Core standard. 

The starting point of this data-publication process is the output of the monthly running data pipeline/ETL of the raw logs towards a processed logs version as it is provided to the researchers. Technical setup explained [here](https://github.com/inbo/bird-tracking-aws) and processing steps available as R code [here](https://github.com/inbo/bird-tracking-etl). The outcome of this ETL is a `.CSV` and a `.sqlite` data output available (for registered users) at http://birdtracking-downloader.eu-west-1.elasticbeanstalk.com/. A download from the `csv` version of one of the outputs can be used here to convert it to a Darwin Core.

In [179]:
import os
import gzip
import shutil

import pandas as pd

For the reference, providing the sqlite version of the data-selection:

## A `csv` version of the raw logs - subset check

In [180]:
path = '2017-04-03-processed-logs.csv.gz'
if os.path.isfile(path.rstrip('.gz')):
    print("raw csv log file available for analysis")
elif os.path.isfile(path):
    with gzip.open(path, 'rb') as src, open(path.rstrip('.gz'), 'wb') as dest:
            shutil.copyfileobj(src, dest)
    os.remove(path)
    print("csv file extracted from zip-folder")
else:
    print("No csv or zip available, pleas provide file.")

raw csv log file available for analysis


Subset of data reading with Pandas, all as string object -> 50000 records

In [181]:
data = pd.read_csv(path.rstrip('.gz'), nrows=50000, dtype=object)

In [182]:
data.columns

Index(['calc_corine_value', 'device_info_serial', 'project_leader',
       'bird_name', 'ring_code', 'colour_ring_code', 'species_code',
       'scientific_name', 'catch_weight', 'sex', 'catch_location',
       'tracking_started_at', 'tracking_ended_at', 'is_active', 'remarks',
       'colony_latitude', 'colony_longitude', 'date_time', 'latitude',
       'longitude', 'altitude', 'pressure', 'temperature', 'satellites_used',
       'gps_fixtime', 'positiondop', 'h_accuracy', 'v_accuracy', 'x_speed',
       'y_speed', 'z_speed', 'speed_accuracy', 'userflag', 'speed_3d',
       'speed_2d', 'direction', 'altitude_agl', 'calc_year', 'calc_month',
       'calc_hour', 'calc_time_diff', 'calc_distance_diff', 'calc_speed_2d',
       'calc_distance_to_colony', 'calc_sunlight', 'calc_outlier',
       'calc_corine_legend'],
      dtype='object')

### A sample dataset for comparison

This dataset is already available from the previous ETL

In [134]:
sample_outcome = pd.read_csv("dwc-occurrence-100.csv")

In [135]:
sample_outcome.head()

occurrenceid   type              modified language  \
0  703:20130530182507  Event  2015-04-29T09:34:49Z       en   
1  703:20130530185523  Event  2015-04-29T09:34:49Z       en   
2  703:20130530192538  Event  2015-04-29T09:34:49Z       en   
3  703:20130530195536  Event  2015-04-29T09:34:49Z       en   
4  703:20130530202528  Event  2015-04-29T09:34:49Z       en   

                                             license rightsholder  \
0  http://creativecommons.org/publicdomain/zero/1.0/         INBO   
1  http://creativecommons.org/publicdomain/zero/1.0/         INBO   
2  http://creativecommons.org/publicdomain/zero/1.0/         INBO   
3  http://creativecommons.org/publicdomain/zero/1.0/         INBO   
4  http://creativecommons.org/publicdomain/zero/1.0/         INBO   

                               accessrights                       datasetid  \
0  http://www.inbo.be/en/norms-for-data-use  http://doi.org/10.15468/02omly   
1  http://www.inbo.be/en/norms-for-data-use  http://doi.org/10.15468/02omly   
2  http://www.inbo.be/en/norms-for-data-use  http://doi.org/10.15468/02omly   
3  http://www.inbo.be/en/norms-for-data-use  http://doi.org/10.15468/02omly   
4  http://www.inbo.be/en/norms-for-data-use  http://doi.org/10.15468/02omly   

  institutioncode                                        datasetname  \
0            INBO  Bird tracking - GPS tracking of Lesser Black-b...   
1            INBO  Bird tracking - GPS tracking of Lesser Black-b...   
2            INBO  Bird tracking - GPS tracking of Lesser Black-b...   
3            INBO  Bird tracking - GPS tracking of Lesser Black-b...   
4            INBO  Bird tracking - GPS tracking of Lesser Black-b...   

         ...           phylum class           _order   family  genus  \
0        ...         Chordata  Aves  Charadriiformes  Laridae  Larus   
1        ...         Chordata  Aves  Charadriiformes  Laridae  Larus   
2        ...         Chordata  Aves  Charadriiformes  Laridae  Larus   
3        ...         Chordata  Aves  Charadriiformes  Laridae  Larus   
4        ...         Chordata  Aves  Charadriiformes  Laridae  Larus   

  specificepithet taxonrank scientificnameauthorship  \
0          fuscus   species           Linnaeus, 1758   
1          fuscus   species           Linnaeus, 1758   
2          fuscus   species           Linnaeus, 1758   
3          fuscus   species           Linnaeus, 1758   
4          fuscus   species           Linnaeus, 1758   

             vernacularname nomenclaturalcode  
0  Lesser Black-backed Gull              ICZN  
1  Lesser Black-backed Gull              ICZN  
2  Lesser Black-backed Gull              ICZN  
3  Lesser Black-backed Gull              ICZN  
4  Lesser Black-backed Gull              ICZN  

[5 rows x 43 columns]

### Python/ic/Pandanic ETL - step by step

For each of the original [sql-statements](./dwc-occurrence.sql), a Python-translation will be setup and the resulting Darwin Core version stored as a DataFrame `df_dwc`:

__Remark__: The dates are already inn UTC, so no time zone shifting is required, dates are directly converted to the requried string representation

    t.device_info_serial || ':' || to_char(t.date_time at time zone 'UTC','YYYYMMDDHH24MISS') as occurrenceID,

In [ ]:
data["date_time"] = pd.to_datetime(data["date_time"]) # read the dates
data["occurrence_ID"] = data["device_info_serial"].astype(str) + ":" +  data["date_time"].dt.strftime("%Y%M%d%H%M%S")
data = data.sort_values(["occurrence_ID", "date_time"])

# setup the New DataFrame, with the occurrence_ID as first column:
df_dwc = pd.DataFrame(data["occurrence_ID"]).copy()

In [183]:
#df_dwc = pd.DataFrame(data["device_info_serial"].astype(str) + ":" +  data["date_time"].dt.strftime("%Y%M%d%H%M%S"), columns=["occurrence_ID"])

A set of static columns:

      'Event'::text as type,
        'en'::text as language,
      'http://creativecommons.org/publicdomain/zero/1.0/'::text as license,
      'INBO'::text as rightsholder,
      'http://www.inbo.be/en/norms-for-data-use'::text as accessRights,
      'http://dataset.inbo.be/bird-tracking-wmh-occurrences'::text as datasetID,
      'INBO'::text as institutionCode,
      'Bird tracking - GPS tracking of Western Marsh Harriers breeding near the Belgium-Netherlands border'::text as datasetName,
      'INBO'::text as ownerInstitutionCode,
      'MachineObservation'::text as basisOfRecord,
      'see metadata'::text as informationWithheld,

In [120]:
df_dwc["type"] = "Event"
df_dwc["language"] = "en"
df_dwc["license"] = "http://creativecommons.org/publicdomain/zero/1.0/"
df_dwc["rightsholder"] = "INBO"
df_dwc["accessRights"] = "http://www.inbo.be/en/norms-for-data-use"
df_dwc["datasetID"] = "http://dataset.inbo.be/bird-tracking-wmh-occurrences"
df_dwc["institutionCode"] = "INBO"
df_dwc["datasetName"] = "Bird tracking - GPS tracking of Western Marsh Harriers breeding near the Belgium-Netherlands border"
df_dwc["ownerInstitutionCode"] = "INBO"
df_dwc["basisOfRecord"] = "MachineObservation"
df_dwc["informationWithheld"] = "see metadata"

The following will not be taken into account:

    to_char(t.updated_at at time zone 'UTC','YYYY-MM-DD"T"HH24:MI:SS"Z"') as modified,  # updated_at is a CARTO thing...

More static columns:

    ('{"device_info_serial":' || t.device_info_serial)::text || '}' as dynamicProperties,
    d.sex::text as sex,
    'adult'::text as lifeStage,
    d.ring_code::text as organismID,
    d.bird_name::text as organismName,
    'doi:10.1007/s10336-012-0908-1'::text as samplingProtocol,

In [121]:
df_dwc["dynamicProperties"] = "{'device_info_serial':" + data["device_info_serial"].astype(str) + "}"
df_dwc["sex"] = data["sex"]
df_dwc["lifeStage"] = "adult"
df_dwc["organismID"] = data["ring_code"]
df_dwc["organismName"] = data["bird_name"]
df_dwc["samplingProtocol"] = "doi:10.1007/s10336-012-0908-1"

In [122]:
df_dwc

occurrence_ID   type language  \
1959   610:20130001000038  Event       en   
1960   610:20130001010034  Event       en   
1961   610:20130001020051  Event       en   
1962   610:20130001030050  Event       en   
2204   610:20130007230002  Event       en   
2206   610:20130008010033  Event       en   
2207   610:20130008020031  Event       en   
2208   610:20130008030031  Event       en   
1181   610:20130008230002  Event       en   
1182   610:20130009000000  Event       en   
1183   610:20130009010002  Event       en   
1184   610:20130009020032  Event       en   
1185   610:20130009030046  Event       en   
1186   610:20130009040057  Event       en   
1207   610:20130009160048  Event       en   
1209   610:20130009170045  Event       en   
1213   610:20130009190046  Event       en   
1214   610:20130009200046  Event       en   
1215   610:20130009210046  Event       en   
1225   610:20130010070046  Event       en   
1227   610:20130010080032  Event       en   
1231   610:20130010100046  Event       en   
1233   610:20130010110044  Event       en   
275    610:20130014120054  Event       en   
2534   610:20130017070001  Event       en   
2536   610:20130017080010  Event       en   
2538   610:20130017090006  Event       en   
2540   610:20130017100002  Event       en   
2542   610:20130017110022  Event       en   
2544   610:20130017120009  Event       en   
...                   ...    ...      ...   
43141  833:20145925145920  Event       en   
43145  833:20145925155911  Event       en   
48094  833:20145926225909  Event       en   
48110  833:20145927045923  Event       en   
48114  833:20145927055907  Event       en   
48126  833:20145927085907  Event       en   
43282  833:20145927085956  Event       en   
48130  833:20145927095924  Event       en   
48134  833:20145927105915  Event       en   
43290  833:20145927105957  Event       en   
48138  833:20145927115934  Event       en   
48142  833:20145927125957  Event       en   
48175  833:20145927225909  Event       en   
48199  833:20145928065911  Event       en   
48203  833:20145928075911  Event       en   
48207  833:20145928085923  Event       en   
48211  833:20145928095909  Event       en   
48215  833:20145928105942  Event       en   
48219  833:20145928115924  Event       en   
45940  833:20145928165938  Event       en   
43420  833:20145929005914  Event       en   
43422  833:20145929015938  Event       en   
43424  833:20145929025919  Event       en   
43428  833:20145929035934  Event       en   
43432  833:20145929045954  Event       en   
43435  833:20145929055952  Event       en   
48371  833:20145930085935  Event       en   
48375  833:20145930095958  Event       en   
43596  833:20145931045901  Event       en   
43600  833:20145931055955  Event       en   

                                                 license rightsholder  \
1959   http://creativecommons.org/publicdomain/zero/1.0/         INBO   
1960   http://creativecommons.org/publicdomain/zero/1.0/         INBO   
1961   http://creativecommons.org/publicdomain/zero/1.0/         INBO   
1962   http://creativecommons.org/publicdomain/zero/1.0/         INBO   
2204   http://creativecommons.org/publicdomain/zero/1.0/         INBO   
2206   http://creativecommons.org/publicdomain/zero/1.0/         INBO   
2207   http://creativecommons.org/publicdomain/zero/1.0/         INBO   
2208   http://creativecommons.org/publicdomain/zero/1.0/         INBO   
1181   http://creativecommons.org/publicdomain/zero/1.0/         INBO   
1182   http://creativecommons.org/publicdomain/zero/1.0/         INBO   
1183   http://creativecommons.org/publicdomain/zero/1.0/         INBO   
1184   http://creativecommons.org/publicdomain/zero/1.0/         INBO   
1185   http://creativecommons.org/publicdomain/zero/1.0/         INBO   
1186   http://creativecommons.org/publicdomain/zero/1.0/         INBO   
1207   http://creativecommons.org/publicdomain/zero/1.0/         INBO   
1209   http://creativecommons.org/publicdomain/zero/1

A case defining  the interval in seconds:

    case
        when intervalinseconds >= 0 then ('{"secondsSinceLastOccurrence":' || intervalinseconds || '}')::text
        else '{"secondsSinceLastOccurrence":}'::text
    end as samplingEffort,

In [123]:
def samplingeff(x):
    if x > 0.:
        return "{'secondsSinceLastOccurrence':" + str(round(x)) +  "}"
    else:
        return "{'secondsSinceLastOccurrence':}"

In [124]:
df_dwc["intervalinseconds"] = data["date_time"].diff(1).dt.seconds
df_dwc["samplingEffort"] = df_dwc["intervalinseconds"].apply(samplingeff)
df_dwc = df_dwc.drop("intervalinseconds", axis=1)

Defining the datetime (not timezone changes provided):

    to_char(t.date_time at time zone 'UTC','YYYY-MM-DD"T"HH24:MI:SS"Z"') as eventDate,

In [125]:
df_dwc["eventDate"] = data["date_time"].dt.strftime("%Y-%M-%dT%H:%M:%SZ")

Direct conversions and static values:
  
    0::numeric as minimumElevationInMeters,
    t.altitude::numeric as minimumDistanceAboveSurfaceInMeters,
    t.latitude::numeric as decimalLatitude,
    t.longitude::numeric as decimalLongitude,
    'WGS84'::text as geodeticDatum,
    
**Remark**: no data type conversion required, direct copy of the string representation of the data:

In [126]:
df_dwc["minimumElevationInMeters"] = "0"
df_dwc["minimumDistanceAboveSurfaceInMeters"] = data["altitude"]
df_dwc["decimalLatitude"] = data["latitude"]
df_dwc["decimalLongitude"] = data["longitude"]
df_dwc["geodeticDatum"] = "WGS84"

Defining the uncertainty of the measurement  
  
    case
        when h_accuracy is not null then round(h_accuracy)::numeric
        else 30::numeric
    end as coordinateUncertaintyInMeters,

In [127]:
data["h_accuracy"] = data["h_accuracy"].astype(float)
df_dwc["coordinateUncertaintyInMeters"] = round(data["h_accuracy"].where(data["h_accuracy"].notnull(), 30)).astype(int)

Another datetim string print:

    to_char(t.date_time at time zone 'UTC','YYYY-MM-DD"T"HH24:MI:SS"Z"') as georeferencedDate,

In [128]:
df_dwc["georeferencedDate"] = data["date_time"].dt.strftime("%Y-%M-%dT%H:%M:%SZ")

Static values:

    'doi:10.1080/13658810412331280211'::text as georeferenceProtocol,
    'GPS'::text as georeferenceSources,
    'unverified'::text as georeferenceVerificationStatus,

In [129]:
df_dwc["georeferenceProtocol"] = 'doi:10.1080/13658810412331280211'
df_dwc["georeferenceSources"] = 'GPS'
df_dwc["georeferenceVerificationStatus"] = 'unverified'

    d.scientific_name::text as scientificName,
    'Animalia'::text as kingdom,
    'Chordata'::text as phylum,
    'Aves'::text as class,
    'Charadriiformes'::text as _order,
    'Laridae'::text as family,

In [130]:
df_dwc["scientificName"] = data["scientific_name"]
df_dwc["kingdom"] = 'Animalia'
df_dwc["phylum"] = 'Chordata'
df_dwc["class"] = 'Aves'
df_dwc["order"] = 'Charadriiformes'
df_dwc["family"] = 'Laridae'
df_dwc["taxonRank"] = 'species'

  Get the genus and specificEpithet out of the scientific name:
      
      split_part(d.scientific_name, ' ', 1)::text as genus,
      split_part(d.scientific_name, ' ', 2)::text as specificEpithet

In [131]:
df_dwc[["genus", "specificEpithet"]] = data["scientific_name"].str.split().apply(pd.Series)

Authorship and vernaculuar name for the Gulls:

    case
        when d.species_code = 'lbbg' then 'Linnaeus, 1758'
        when d.species_code = 'hg' then 'Pontoppidan, 1763'
    end::text as scientificNameAuthorship,
    case
        when d.species_code = 'lbbg' then 'Lesser Black-backed Gull'
        when d.species_code = 'hg' then 'Herring Gull'
    end::text as vernacularName,
    'ICZN'::text as nomenclaturalCode

In [132]:
df_dwc["scientificNameAuthorship"] = data["species_code"].replace(to_replace=["lbbg", "hg"], value=["Linnaeus, 1758", "Pontoppidan, 1763"])
df_dwc["vernacularName"] = data["species_code"].replace(to_replace=["lbbg", "hg"], value=["Lesser Black-backed Gull", "Herring Gull"])
df_dwc["nomenclaturalCode"] = "ICZN"

In [177]:
df_dwc.head()

occurrence_ID   type language  \
1959  610:20130001000038  Event       en   
1960  610:20130001010034  Event       en   
1961  610:20130001020051  Event       en   
1962  610:20130001030050  Event       en   
2204  610:20130007230002  Event       en   

                                                license rightsholder  \
1959  http://creativecommons.org/publicdomain/zero/1.0/         INBO   
1960  http://creativecommons.org/publicdomain/zero/1.0/         INBO   
1961  http://creativecommons.org/publicdomain/zero/1.0/         INBO   
1962  http://creativecommons.org/publicdomain/zero/1.0/         INBO   
2204  http://creativecommons.org/publicdomain/zero/1.0/         INBO   

                                  accessRights  \
1959  http://www.inbo.be/en/norms-for-data-use   
1960  http://www.inbo.be/en/norms-for-data-use   
1961  http://www.inbo.be/en/norms-for-data-use   
1962  http://www.inbo.be/en/norms-for-data-use   
2204  http://www.inbo.be/en/norms-for-data-use   

                                              datasetID institutionCode  \
1959  http://dataset.inbo.be/bird-tracking-wmh-occur...            INBO   
1960  http://dataset.inbo.be/bird-tracking-wmh-occur...            INBO   
1961  http://dataset.inbo.be/bird-tracking-wmh-occur...            INBO   
1962  http://dataset.inbo.be/bird-tracking-wmh-occur...            INBO   
2204  http://dataset.inbo.be/bird-tracking-wmh-occur...            INBO   

                                            datasetName ownerInstitutionCode  \
1959  Bird tracking - GPS tracking of Western Marsh ...                 INBO   
1960  Bird tracking - GPS tracking of Western Marsh ...                 INBO   
1961  Bird tracking - GPS tracking of Western Marsh ...                 INBO   
1962  Bird tracking - GPS tracking of Western Marsh ...                 INBO   
2204  Bird tracking - GPS tracking of Western Marsh ...                 INBO   

            ...           phylum class            order   family taxonRank  \
1959        ...         Chordata  Aves  Charadriiformes  Laridae   species   
1960        ...         Chordata  Aves  Charadriiformes  Laridae   species   
1961        ...         Chordata  Aves  Charadriiformes  Laridae   species   
1962        ...         Chordata  Aves  Charadriiformes  Laridae   species   
2204        ...         Chordata  Aves  Charadriiformes  Laridae   species   

       genus specificEpithet scientificNameAuthorship vernacularName  \
1959  Circus     aeruginosus                      wmh            wmh   
1960  Circus     aeruginosus                      wmh            wmh   
1961  Circus     aeruginosus                      wmh            wmh   
1962  Circus     aeruginosus                      wmh            wmh   
2204  Circus     aeruginosus                      wmh            wmh   

     nomenclaturalCode  
1959              ICZN  
1960              ICZN  
1961              ICZN  
1962              ICZN  
2204              ICZN  

[5 rows x 42 columns]

### Pythonic ETL - function

TODO, when revision of the conversion is done...